In [ ]:
%load_ext autoreload
%autoreload 2

In [69]:
from modelTuner import loadDummyData
from src.modelTraining.testingModels import SelfAttentionNet, default_params_sattention

In [46]:
dataloader = loadDummyData(batch_size=64)

/Users/mgrapotte/LabWork/LearnTF/src/datasetSimulation/TFFamilyClass.py:71: RuntimeWarning: divide by zero encountered in log2
  m = np.log2(ppm.T/bg)


### Run after this line (dataloader is slow)

In [70]:
model = SelfAttentionNet(params=default_params_sattention)

In [71]:
inputs = next(iter((dataloader)))

In [72]:
DNA = inputs[0].squeeze()

In [73]:
first_out = model(DNA)[0]

### Train Loop

In [78]:
### Config setting
config = {
    'modelconfig' : default_params_sattention,
    "lr": 1e-2,
    "loss": "mse",
    "optimizer":"adam"}  

In [76]:
def train(config, trainloader):
    
    # initialize model 
    net = convDNA(config["modelconfig"])

    # send model to the right device
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    # setup optimizer and loss according to config
    if config["optimizer"] == "adam":
        optimizer = torch.optim.Adam(net.parameters(), lr=config["lr"])
    elif config["optimizer"] == "sgd":
        optimizer = torch.optim.SGD(net.parameters(), lr=config["lr"])

    if config["loss"] == "mse":
        criterion = nn.MSELoss()
    elif config["loss"] == "mae":
        criterion = nn.L1Loss()
    elif config["loss"] == "smooth_l1":
        criterion = nn.SmoothL1Loss()

    # The `checkpoint_dir` parameter gets passed by Ray Tune when a checkpoint
    # should be restored.
    if checkpoint_dir:
        checkpoint = os.path.join(checkpoint_dir, "checkpoint")
        model_state, optimizer_state = torch.load(checkpoint)
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    # train model
    for epoch in range(50):
        train_loss = 0.0
        train_steps = 0
        pred = []
        true = []
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            dna, prot, labels = data
            dna, prot, labels = dna.to(device), prot.to(device), labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = net(dna)
            true += labels.tolist()
            pred += outputs.tolist()
            loss = criterion(outputs, labels.view(-1,1).float())
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_steps += 1

        with tune.checkpoint_dir(step=epoch) as checkpoint_dir:
                    path = os.path.join(checkpoint_dir, "checkpoint")
                    torch.save(
                        (net.state_dict(), optimizer.state_dict()), path)
        tune.report(loss=(train_loss / train_steps))# , accuracy=_roc_accuracy(true, pred, 5)[0])